In [188]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import torch.optim as optim

np.random.seed(8)

In [194]:
classificator = pd.read_csv('hobby_data/Classificator_hachaton.csv', delimiter=';')
print(classificator.shape)
classificator.head()

(623, 3)


,id_классификатора,id_родительского_классификатора,Наименование
0,3002985,NaN,Автошкола
1,3002986,3002985.0,Автоклуб
2,3002987,3002985.0,Правила дорожного движения
3,3002988,3002985.0,Юный водитель
4,3002989,NaN,Архитектура


In [195]:
classificator['id_родительского_классификатора'].isin(classificator['id_классификатора']).sum()

585

In [196]:
classificator['Наименование'].unique().shape

(517,)

In [197]:
for idx, row in classificator.iterrows():
    print(row['Наименование'])
    print(classificator[classificator['id_классификатора'] == row['id_родительского_классификатора']]['Наименование'].to_list())
    print()
    if idx > 2:
        break

Автошкола
[]

Автоклуб
['Автошкола']

Правила дорожного движения
['Автошкола']

Юный водитель
['Автошкола']



In [198]:
megaRelation = pd.read_csv('hobby_data/MegaRelation_hackaton.csv', delimiter=';')
print(megaRelation.shape)
megaRelation.head()

(724813, 14)


,id_зачисления,Статус,id_ученика,Дата_создания_записи,id_организации,id_заявления,id_услуги,дата_зачисления,дата_отчисления,причина_перевода,предыдущая_запись_зачисления,следующая_запись_зачисления,Плановая_дата_начала_занятий,Плановая_дата_окончания_занятий
0,1325954,3.0,25969.0,08.06.2015 14:47:19,30,950832.0,39178,08.06.2015 0:00:00,29.02.2016 19:28:12,NaN,NaN,3558114.0,01.09.2014 0:00:00,01.09.2021 0:00:00
1,2879918,3.0,158129.0,29.10.2015 18:35:21,30,989658.0,39178,01.09.2010 0:00:00,29.02.2016 19:23:59,NaN,NaN,3557885.0,01.09.2014 0:00:00,01.09.2021 0:00:00
2,2901258,3.0,304476.0,02.11.2015 14:05:56,30,950764.0,39178,09.06.2014 0:00:00,29.02.2016 19:28:11,NaN,NaN,3558103.0,01.09.2014 0:00:00,01.09.2021 0:00:00
3,2912724,3.0,1139915.0,03.11.2015 14:29:27,30,950786.0,39178,09.06.2014 0:00:00,29.02.2016 19:28:11,NaN,NaN,3558105.0,01.09.2014 0:00:00,01.09.2021 0:00:00
4,2912819,3.0,298767.0,03.11.2015 14:34:15,30,950799.0,39178,09.06.2014 0:00:00,29.02.2016 19:28:11,NaN,NaN,3558106.0,01.09.2014 0:00:00,01.09.2021 0:00:00


In [199]:
megaRelation[megaRelation['id_зачисления'] == 3558114.0]

,id_зачисления,Статус,id_ученика,Дата_создания_записи,id_организации,id_заявления,id_услуги,дата_зачисления,дата_отчисления,причина_перевода,предыдущая_запись_зачисления,следующая_запись_зачисления,Плановая_дата_начала_занятий,Плановая_дата_окончания_занятий
387310,3558114,3.0,25969.0,29.02.2016 19:28:12,30,950832.0,227141,29.02.2016 19:28:12,11.12.2018 19:15:29,NaN,1325954.0,17660387.0,01.09.2014 0:00:00,01.09.2021 0:00:00


In [200]:
services[services['id_услуги'] == 39178]

,id_услуги,Тип_финансирования,Классификатор_услуги,id_организации,Тип_расписания,Наименование_услуги,Дата_создания,длительность_обучения,единица_длительности
0,39178,бесплатно,3000001,30,Индивидуальное,Архитектура,17.07.2013 12:50:32,"7,0000",лет


In [201]:
services[services['id_услуги'] == 227141]

,id_услуги,Тип_финансирования,Классификатор_услуги,id_организации,Тип_расписания,Наименование_услуги,Дата_создания,длительность_обучения,единица_длительности
21085,227141,бесплатно,3210100,30,Общее,Архитектура. Общеразвивающая общеобразовательн...,26.02.2016 12:40:25,"7,0000",лет


In [202]:
megaRelation.isna().sum(axis=0)

id_зачисления                           0
Статус                              77036
id_ученика                          77036
Дата_создания_записи                    1
id_организации                          0
id_заявления                        41913
id_услуги                               0
дата_зачисления                     41526
дата_отчисления                    239720
причина_перевода                   724811
предыдущая_запись_зачисления       514737
следующая_запись_зачисления        526148
Плановая_дата_начала_занятий          393
Плановая_дата_окончания_занятий      4916
dtype: int64

In [203]:
megaRelation[megaRelation['предыдущая_запись_зачисления'].isna()].shape

(514737, 14)

In [204]:
megaRelation['Статус'].value_counts()

2.0    288103
3.0    198300
1.0    161374
Name: Статус, dtype: int64

In [205]:
megaRelation['id_ученика'].unique().shape

(289170,)

In [206]:
megaRelation['id_организации'].unique().shape

(238,)

In [207]:
megaRelation['id_заявления'].unique().shape

(473097,)

In [208]:
megaRelation['причина_перевода'].value_counts()

2.0    2
Name: причина_перевода, dtype: int64

In [209]:
megaRelation['Статус'].value_counts()

2.0    288103
3.0    198300
1.0    161374
Name: Статус, dtype: int64

In [210]:
megaRelation['id_заявления'].unique().shape

(473097,)

In [211]:
(~request['id_заявления'].isin(megaRelation['id_заявления'])).sum()

NameError: name 'request' is not defined

In [212]:
(~megaRelation['id_заявления'].isin(request['id_заявления'])).sum()

NameError: name 'request' is not defined

In [213]:
megaRelation['id_заявления'].value_counts().describe([0.95])

count    473096.000000
mean          1.443470
std           6.166556
min           1.000000
50%           1.000000
95%           3.000000
max        1226.000000
Name: id_заявления, dtype: float64

In [214]:
megaRelation[megaRelation['id_заявления'] == 4245658.0][:3]

,id_зачисления,Статус,id_ученика,Дата_создания_записи,id_организации,id_заявления,id_услуги,дата_зачисления,дата_отчисления,причина_перевода,предыдущая_запись_зачисления,следующая_запись_зачисления,Плановая_дата_начала_занятий,Плановая_дата_окончания_занятий
66540,2996534,2.0,13577.0,13.11.2015 19:03:50,59,4245658.0,44945,13.11.2015 19:03:50,14.06.2016 20:26:28,NaN,11441.0,NaN,13.11.2015 19:03:50,13.11.2022 19:03:50
66541,2996535,2.0,13580.0,13.11.2015 19:03:50,59,4245658.0,44945,13.11.2015 19:03:50,14.06.2016 20:26:28,NaN,11444.0,NaN,13.11.2015 19:03:50,13.11.2022 19:03:50
66542,2996536,2.0,13583.0,13.11.2015 19:03:50,59,4245658.0,44945,13.11.2015 19:03:50,14.06.2016 20:11:39,NaN,11447.0,NaN,13.11.2015 19:03:50,13.11.2022 19:03:50


In [215]:
request[request['id_заявления'] == 4245658.0]

NameError: name 'request' is not defined

In [216]:
4245658 in request['id_заявления'].to_list()

NameError: name 'request' is not defined

In [217]:
import datetime as dt
import numpy as np

megaRelation[megaRelation['id_заявления'] == 4245658.0]['Дата_создания_записи'].apply(lambda x: dt.datetime.strptime(x, '%d.%m.%Y %H:%M:%S') if not isinstance(x, float) else np.nan).describe()


count                    1226
unique                    609
top       2015-12-09 12:09:14
freq                       13
first     2015-11-13 19:03:50
last      2019-11-09 20:44:59
Name: Дата_создания_записи, dtype: object

In [218]:
services = pd.read_csv('hobby_data/services_hackaton.csv', delimiter=';')
print(services.shape)
services.head()

(40972, 9)


,id_услуги,Тип_финансирования,Классификатор_услуги,id_организации,Тип_расписания,Наименование_услуги,Дата_создания,длительность_обучения,единица_длительности
0,39178,бесплатно,3000001,30,Индивидуальное,Архитектура,17.07.2013 12:50:32,"7,0000",лет
1,39181,бесплатно,3000001,30,Индивидуальное,Архитектура,17.07.2013 13:04:41,"7,0000",лет
2,212072,бесплатно,3000001,30,Индивидуальное,Архитектура,27.10.2015 15:59:48,"7,0000",лет
3,212517,бесплатно,3000001,30,Индивидуальное,Архитектура,29.10.2015 18:45:49,"7,0000",лет
4,39913,бесплатно,3000003,45,Индивидуальное,Декоративно-прикладное искусство,17.07.2013 20:35:28,"7,0000",лет


In [219]:
services.merge(classificator, left_on='Классификатор_услуги', right_on='id_классификатора').shape

(35867, 12)

In [220]:
services[services['Классификатор_услуги'].isin(classificator['id_классификатора'])].shape

(35867, 9)

In [221]:
services['Наименование_услуги'].unique().shape

(17057,)

In [222]:
services['Наименование_услуги'].unique()[:10]

array(['Архитектура', 'Декоративно-прикладное искусство',
       'Компьютерный дизайн (комплексная  3-летняя программа)',
       'Изобразительное искусство (комплексная 4-летняя программа)',
       'Изобразительное искусство', 'Компьютерная графика и анимация',
       'Художественная фотография',
       'Академическое сольное пение (комплексная  5-летняя программа)',
       'Академическое сольное пение (комплексная  7-летняя программа)',
       nan], dtype=object)

In [223]:
services['единица_длительности'].unique()

array(['лет', 'недель', 'месяцев', 'дней', nan], dtype=object)

In [224]:
services['Тип_финансирования'].value_counts()

бесплатно    28791
платно       12181
Name: Тип_финансирования, dtype: int64

In [225]:
services['Тип_расписания'].value_counts()

Общее             26630
Индивидуальное    14341
Name: Тип_расписания, dtype: int64

In [226]:
org = pd.read_csv('hobby_data/org_hackaton.csv', delimiter=';')
print(org.shape)
org.head()

(443, 5)


,id_организации,полное_наименование,краткое_наименование,улица,Метро
0,1,Государственное бюджетное образовательное учре...,ГБОУ СПО г.Москвы «Московская средняя специаль...,NaN,NaN
1,2,Государственное бюджетное профессиональное обр...,"ГБПОУ г. Москвы ""Академия джаза""",улица Трёхгорный Вал,Краснопресненская
2,2,Государственное бюджетное профессиональное обр...,"ГБПОУ г. Москвы ""Академия джаза""",улица Трёхгорный Вал,Улица 1905 года
3,3,Государственное бюджетное образовательном учре...,ГБОУ СПО г.Москвы «Московский театральный колл...,улица Чаплыгина,Красные ворота
4,4,Государственное бюджетное образовательное учре...,ГБОУДОД г.Москвы «МГДМШ им. С.С.Прокофьева»,Токмаков переулок,Курская (Кольцевая)


In [227]:
org['полное_наименование'].unique().shape

(299,)

In [228]:
org['Метро'].unique().shape

(154,)

In [229]:
pupil = pd.read_csv('hobby_data/Pupil_hackaton.csv', delimiter=';')
print(pupil.shape)
pupil.head()

(2742631, 3)


,id_ученика,возраст,пол
0,1,12.0,Женский
1,2,12.0,Женский
2,3,12.0,Женский
3,4,11.0,Женский
4,5,12.0,Женский


In [230]:
request[set(request.columns) - set(['дата_создания', 'id_услуги', 'id_заявления'])].drop_duplicates().shape

NameError: name 'request' is not defined

In [231]:
request = pd.read_csv('hobby_data/request_hackaton.csv', delimiter=';')
print(request.shape)
request.head()

(470599, 9)


,id_заявления,статус_заявления,дата_создания,оценка_вступительных_испытаний,Испытания_пройдены,Зачисление_разрешено,Зачисление_не_разрешено,Причина_отказа_в_зачислении,id_услуги
0,989578,Ожидание прихода Заявителя на вступительные ис...,02.06.2014 17:25:37,NaN,0.0,0.0,0.0,NaN,39178
1,3571456,Архив,05.11.2015 18:51:59,5.0,1.0,0.0,NaN,NaN,39178
2,3571725,Отказ в зачислении,05.11.2015 19:35:39,NaN,0.0,0.0,1.0,NaN,39178
3,950963,Ожидание прихода Заявителя на вступительные ис...,12.05.2014 16:06:20,NaN,0.0,0.0,0.0,NaN,39178
4,952962,Ожидание прихода Заявителя на вступительные ис...,14.05.2014 18:14:47,NaN,0.0,0.0,0.0,NaN,39178


In [232]:
request.isna().sum(axis=0)

id_заявления                           0
статус_заявления                       0
дата_создания                          0
оценка_вступительных_испытаний    447606
Испытания_пройдены                145464
Зачисление_разрешено              105264
Зачисление_не_разрешено           246739
Причина_отказа_в_зачислении       469321
id_услуги                              0
dtype: int64

In [233]:
request['оценка_вступительных_испытаний'].value_counts()

4.0    14035
5.0     8014
3.0      903
2.0       41
Name: оценка_вступительных_испытаний, dtype: int64

In [234]:
request['Испытания_пройдены'].value_counts()

0.0    302023
1.0     23112
Name: Испытания_пройдены, dtype: int64

In [235]:
request['статус_заявления'].value_counts()

Зачислен                                                 309286
Ожидание прихода Заявителя на вступительные испытания    110096
Ожидание прихода Заявителя для заключения договора        46172
Черновик                                                   1720
Отказ в зачислении                                         1296
Отчислен                                                    905
Вступительные испытания пройдены                            877
Архив                                                       235
Поданы документы                                             12
Name: статус_заявления, dtype: int64

In [236]:
print(request['Причина_отказа_в_зачислении'].isna().sum())
request['Зачисление_не_разрешено'].value_counts()

469321


0.0    222543
1.0      1317
Name: Зачисление_не_разрешено, dtype: int64

In [237]:
request[request['Зачисление_не_разрешено'] == 1].merge(megaRelation, on='id_заявления').shape

(1141, 22)

In [28]:
catalog = pd.read_csv('books_data/catalog.csv')
print(catalog.shape)
catalog.head()

(1370881, 10)


,doc_id,author,title,city,publisher,year,series,tags,knowledge_id,age
0,1,Устинова Татьяна Витальевна,Призрак Канта,Москва,Эксмо,2018,Татьяна Устинова. Первая среди лучших,Художественная литература ; Российский детектив,84(2Рос)6,16+
1,2,NaN,Наука и жизнь,Москва,АНО Редакция журнала «Наука и жизнь»,1938-,NaN,Наука. Науковедение,72,12+
2,3,NaN,Наука и жизнь,NaN,NaN,2018,NaN,Наука. Науковедение,72,12+
3,4,NaN,Наука и жизнь,NaN,NaN,2018,NaN,Наука. Науковедение,72,12+
4,5,NaN,Наука и жизнь,NaN,NaN,2018,NaN,Наука. Науковедение,72,12+


In [8]:
import gensim.downloader as api

w2v = api.load("word2vec-ruscorpora-300")

for n in w2v.most_similar(positive=[u'художник_NOUN'], topn=10):
    print (n[0], n[1])

живописец_NOUN 0.8089516162872314
скульптор_NOUN 0.7210062742233276
фортуни_NOUN 0.6959877610206604
сезаннизм_NOUN 0.671466052532196
остроумова::лебедева_NOUN 0.6667642593383789
неменский_ADJ 0.6574506163597107
жанрист_NOUN 0.6545982360839844
франсиско::гойя_NOUN 0.6541438102722168
портретист_NOUN 0.6540499329566956
плакатист_NOUN 0.6533808708190918


In [10]:
#w2v.get_vector('художник_NOUN')
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger_ru')
#nltk.download('popular')
nltk.download('universal_tagset')
sentence = 'утром мама мыла раму'
sentence = nltk.word_tokenize(sentence)
nltk.tag.pos_tag(sentence,  tagset='universal')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/konstantinlipkin/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


[('утром', 'ADJ'), ('мама', 'NOUN'), ('мыла', 'NOUN'), ('раму', 'NOUN')]

In [7]:
examples = pd.read_csv('books_data/examples.csv')
print(examples.shape)
examples.head()

(11420807, 7)


,issue_id,reader_id,issue_date,inventory_id,barcode,return_date,condition
0,1,179,2018-04-13 15:00:00,03:0000005767,381000177719,2018-04-13 15:00:00,6545
1,2,179,2018-04-13 15:00:00,03:0000005761,381000196420,2018-04-16 15:00:00,6548
2,3,179,2018-04-13 15:00:00,03:0000005767,381000177719,2018-04-13 15:00:00,6545
3,4,179,2018-04-13 15:00:00,03:0000005767,381000177719,2018-04-13 15:00:00,6545
4,5,179,2018-04-13 15:00:00,03:0000005767,381000177719,2018-04-13 15:00:00,6548


In [8]:
readers = pd.read_csv('books_data/readers.csv')
print(readers.shape)
readers.head()

(562133, 2)


,reader_id,birth_date
0,366670,1974-10-14 00:00:00
1,235264,1978-06-01 00:00:00
2,133225,1976-01-06 00:00:00
3,556716,1989-01-28 00:00:00
4,52086,1955-08-14 00:00:00


In [125]:
from pymystem3 import Mystem
import re
class TextPreprocessing:
    def __init__(self):
        self.m = Mystem()
        
    def __lower(self, text):
        return text.lower()
        
    def __clear_text(self, text):
        text = re.sub('[,."!;?]', '', text)
        text = ' '.join(text.split())
        return text

    def __lemmatize(self, text):
        text = self.m.lemmatize(text)[:-1]
        text = ''.join(text)
        return text
    
    def __add_pos_tag(self, text):
        text = nltk.word_tokenize(text)
        text = nltk.tag.pos_tag(text, lang='rus', tagset='universal')
        text = ' '.join([word + '_' + tag for word, tag in text])
        return text

    def __call__(self, text):
        if text is None:
            return ''
        text = self.__lower(text)
        text = self.__clear_text(text)
        text = self.__lemmatize(text)
        text = self.__add_pos_tag(text)
        return text

In [374]:
tp = TextPreprocessing()
tp('Устинова Татьяна Витальевна Призрак Канта Татьяна Устинова. Первая среди лучших Художественная литература ; Российский детектив')

'устинов_NOUN татьяна_NOUN витальевна_ADJ призрак_NOUN кант_NOUN татьяна_NOUN устинов_NOUN первый_ADJ среди_ADP хороший_ADJ художественный_ADJ литература_NOUN российский_ADJ детектив_NOUN'

In [375]:
books = pd.read_csv('books.csv')
print(books.shape)
books.head()

(1587814, 16)


,record_id,sigly,author,title,city,publisher,year,series,tags,knowledge,age,reader_id,issue_date,return_date,condition,birth_date
0,1,314.0,Устинова Татьяна Витальевна,Призрак Канта,Москва,Эксмо,2018,Татьяна Устинова. Первая среди лучших,Художественная литература ; Российский детектив,84(2Рос)6,16+,375196,2019-12-28 15:00:00,2020-01-31 15:00:00,6548,1947-01-02 00:00:00
1,2259,314.0,Устинова Татьяна Витальевна,Селфи с судьбой,Москва,Эксмо,2017,Татьяна Устинова. Первая среди лучших,Художественная литература,84(2Рос)6,16+,375196,2019-12-28 15:00:00,2020-01-31 15:00:00,6548,1947-01-02 00:00:00
2,9234,314.0,Устинова Татьяна Витальевна,Земное притяжение,Москва,Эксмо,2017,Татьяна Устинова. Первая среди лучших,Художественная литература ; Российский детектив,84(2Рос)6,16+,375196,2019-12-28 15:00:00,2020-01-31 15:00:00,6548,1947-01-02 00:00:00
3,18118,314.0,Улицкая Людмила Евгеньевна,Лестница Якова,Москва,АСТ ; Редакция Елены Шубиной,2015,Новая Улицкая,Художественная литература,84(2Рос)6,18+,375196,2019-12-28 15:00:00,2020-01-31 15:00:00,6548,1947-01-02 00:00:00
4,24189,314.0,Рубина Дина Ильинична,Бабий ветер,Москва,Эксмо,2017,Большая проза Дины Рубиной,Художественная литература ; Российская проза,84(2Рос)6,18+,375196,2020-02-12 15:00:00,2020-03-20 15:00:00,6548,1947-01-02 00:00:00


In [376]:
data = catalog[['doc_id', 'author', 'title', 'series', 'tags']].fillna('')
data['text'] = data['author'] +' '+ data['title'] +' '+ data['series'] +' '+ data['tags']
data['text'] = data['tags']
for col in ['author', 'title', 'series', 'tags']:
    del data[col]
data.rename(columns={'doc_id': 'id'}, inplace=True)
data = data[data.id.isin(books.record_id)]
print(data.shape)
data.head()

(35141, 2)


,id,text
0,1,Художественная литература ; Российский детектив
31,33,Художественная литература
36,38,История в целом. Всемирная история
37,39,Художественная литература
39,41,Математика


In [377]:
services['id_услуги'].apply(lambda x: x + 10000000).isin(catalog['doc_id']).sum()

0

In [378]:
services_new = services.copy()
services_new['id_услуги'] = services_new['id_услуги'].apply(lambda x: x + 10000000)
data=data.append(services[['id_услуги', 'Наименование_услуги']].rename(columns={'id_услуги': 'id', 'Наименование_услуги': 'text'}))
data.shape

(76113, 2)

In [379]:
from tqdm import tqdm, tqdm_notebook
#tqdm_notebook().pandas()

data['text_processed'] = ''

for row in tqdm(data.itertuples(), total=len(data)):
    #data.at[idx, 'text_processed'] = tp(row[2])
    try:
        data.at[row[0], 'text_processed'] = tp(row[2])
    except:
        print(row.text)

 47%|████▋     | 35478/76113 [00:26<00:27, 1479.89it/s]

nan
nan
nan
nan
nan
nan
nan
nan


 47%|████▋     | 35814/76113 [00:26<00:29, 1357.10it/s]

nan
nan
nan


 48%|████▊     | 36499/76113 [00:27<00:31, 1244.83it/s]

...

 99%|█████████▉| 75423/76113 [00:59<00:00, 1429.40it/s]

nan
nan
nan
nan
nan
nan


 99%|█████████▉| 75714/76113 [00:59<00:00, 1214.56it/s]

nan
nan
nan
nan


100%|██████████| 76113/76113 [00:59<00:00, 1271.90it/s]


In [380]:
data.dropna(inplace=True)
data.shape

(75703, 3)

In [381]:
import numpy as np
X = []
for row in tqdm(data.itertuples(), total=len(data)):
    vecs = []
    for word in nltk.word_tokenize(row.text_processed):
        if word in w2v.vocab:
            vecs.append(w2v.get_vector(word))
        
    vec = np.mean(vecs, axis=0)
    if not isinstance(vec, np.float) and len(vec) == 300:
        X.append(vec)
    else:
        X.append([0] * 300)
X = np.array(X)

  0%|          | 0/75703 [00:00<?, ?it/s]/home/konstantinlipkin/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/konstantinlipkin/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 75703/75703 [00:10<00:00, 6883.44it/s]


In [382]:
X.shape

(75703, 300)

In [399]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=8000, n_jobs=-1)
kmeans.fit(X)

KMeans(n_clusters=8000, n_jobs=-1)

In [400]:
from collections import Counter
Counter(kmeans.labels_)

Counter({130: 20,
         23: 263,
         372: 18,
         48: 227,
         68: 91,
         27: 276,
         1: 15893,
         14: 489,
         166: 102,
         95: 329,
         617: 30,
         176: 123,
         97: 206,
         4: 5406,
         548: 20,
         21: 312,
         12: 523,
         29: 587,
         11: 1044,
         209: 80,
         73: 61,
         31: 181,
         65: 74,
         16: 837,
         17: 870,
         58: 247,
         415: 19,
         10: 656,
         74: 202,
         499: 69,
         83: 56,
         7: 824,
         1602: 9,
         112: 37,
         244: 46,
         111: 159,
         45: 110,
         674: 3,
         1190: 5,
         1989: 2,
         614: 7,
         159: 8,
         6656: 3,
         1774: 2,
         113: 10,
         2154: 2,
         822: 3,
         3300: 2,
         3752: 2,
         5082: 2,
         666: 4,
         1999: 2,
         1945: 2,
         405: 7,
         1761: 8,
         2363: 3

In [401]:
data['label'] = kmeans.labels_
for label in data.label.unique():
    print(label, data[data.label == label].id.isin(services['id_услуги']).sum(), (~data[data.label == label].id.isin(services['id_услуги'])).sum())

130 19 1
23 203 60
372 9 9
48 221 6
68 86 5
27 263 13
1 293 15600
14 475 14
166 97 5
95 313 16
617 28 2
176 113 10
97 192 14
4 5061 345
548 19 1
21 285 27
12 13 510
29 560 27
11 1016 28
209 74 6
73 56 5
31 170 11
65 72 2
16 801 36
17 837 33
58 238 9
415 18 1
10 629 27
74 187 15
499 63 6
83 53 3
7 806 18
1602 8 1
112 34 3
244 42 4
111 148 11
45 109 1
674 2 1
1190 4 1
1989 1 1
614 5 2
159 6 2
6656 2 1
1774 1 1
113 8 2
2154 1 1
822 2 1
3300 1 1
3752 1 1
5082 1 1
666 3 1
1999 1 1
1945 1 1
405 6 1
1761 7 1
2363 2 1
169 2 2
4715 1 1
642 3 1
5736 1 1
223 7 3
1554 6 1
2511 1 1
1505 2 1
221 12 2
2984 1 1
584 2 1
1471 1 1
3202 1 1
1243 2 1
4079 1 1
2304 2 1
4108 1 1
2993 3 1
2006 1 1
1520 2 2
2092 2 1
991 2 1
2126 1 1
1605 4 2
1072 5 1
3104 1 1
1736 2 1
1995 1 1
895 1 1
574 4 1
3543 1 1
2337 1 1
1268 6 1
1344 2 1
3067 1 1
5851 1 1
2497 1 1
2056 2 1
5345 1 1
387 4 2
1244 7 1
211 6 1
2828 1 1
4068 1 1
1123 1 1
46 2 1
3375 4 2
6819 1 1
1033 3 1
748 1 1
3116 1 1
3671 2 1
3319 2 1
766 19 1
390 3 1
28

In [402]:
class RS(nn.Module):
    
    def __init__(self, n_users, n_hobbies, n_features=10):
        super().__init__()
        n_emb = 10
        self.u_embedding = nn.Embedding(n_users + 1, n_emb)
        self.h_embedding = nn.Embedding(n_hobbies + 1, n_emb)
        self.u_flatten = nn.Flatten()
        self.h_flatten = nn.Flatten()
        self.lin1 = nn.Linear(n_features + 2 * n_emb - 2, 256)
        self.lin2 = nn.Linear(256, 64)
        self.lin3 = nn.Linear(64, 2)
        
        self.batchnorm = nn.BatchNorm1d(128)
        self.relu = nn.ReLU()
        
        self.init_parameters()
        
    def init_parameters(self):
        nn.init.uniform_(self.u_embedding.weight, -1.0, 1.0)
        nn.init.uniform_(self.h_embedding.weight, -1.0, 1.0)
        nn.init.uniform_(self.lin1.weight, -1.0, 1.0)
        nn.init.uniform_(self.lin2.weight, -1.0, 1.0)
        nn.init.uniform_(self.lin3.weight, -1.0, 1.0)
        
    def forward(self, x):
        user = x[:, :1, ...]
        user = user.long()
        user = self.u_embedding(user)
        user = self.u_flatten(user)
        
        hobbie = x[:, 1:2, ...]
        hobbie = hobbie.long()
        hobbie = self.h_embedding(hobbie)
        hobbie = self.h_flatten(hobbie)

        x = torch.cat([user, hobbie, x[:, 2:]], dim=1)
        x = x.float()
        
        x = self.lin1(x)
        x = self.relu(x)
        #x = self.batchnorm(x)
        x = self.lin2(x)
        x = self.lin3(x)
        
        return x

In [242]:
megaRelation = megaRelation[megaRelation['Статус'] != 2]
megaRelation.shape

(436710, 14)

In [243]:
request = request[request['Зачисление_не_разрешено'] != 1]
request.shape

(469282, 9)

In [403]:
dataset = megaRelation[['id_ученика', 'id_заявления', 'id_услуги', 'id_организации']].merge(
    request[['id_заявления', 'оценка_вступительных_испытаний', ]], 
    on='id_заявления', 
    how='left'
)
dataset.shape

(442104, 5)

In [404]:
dataset = dataset.merge(
    pupil,
    on='id_ученика',
    how='left'
)
dataset.shape

(442104, 7)

In [405]:
dataset['id_услуги'] = dataset['id_услуги'].apply(lambda x: x + 10000000)

In [406]:
drop_cols = ['оценка_вступительных_испытаний', 'id_заявления', 'id_организации', 'пол']
for col in drop_cols:
    del dataset[col]

In [407]:
books['age'] = books['birth_date'].apply(lambda x: (dt.datetime.now() - dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')).days / 365.25)
dataset = pd.concat([
    dataset,
    books[['reader_id', 'record_id', 'age']].rename(columns={'reader_id': 'id_ученика', 'record_id': 'id_услуги', 'age': 'возраст'})
], axis=0)
dataset.shape

(2029918, 3)

In [408]:
dataset.head()

,id_ученика,id_услуги,возраст
0,25969.0,10039178,14.0
1,158129.0,10039178,18.0
2,304476.0,10039178,14.0
3,1139915.0,10039178,14.0
4,298767.0,10039178,13.0


In [409]:
dataset = dataset.merge(data, left_on='id_услуги', right_on='id')
dataset.head()

,id_ученика,id_услуги,возраст,id,text,text_processed,label
0,375196.0,1,73.831622,1,Художественная литература ; Российский детектив,архитектура_NOUN,130
1,416672.0,1,41.026694,1,Художественная литература ; Российский детектив,архитектура_NOUN,130
2,349736.0,1,76.038330,1,Художественная литература ; Российский детектив,архитектура_NOUN,130
3,379610.0,1,69.232033,1,Художественная литература ; Российский детектив,архитектура_NOUN,130
4,378839.0,1,54.480493,1,Художественная литература ; Российский детектив,архитектура_NOUN,130


In [410]:
drop_cols = ['id', 'text', 'text_processed', 'id_услуги']
for col in drop_cols:
    del dataset[col]

In [411]:
dataset.rename(columns={'label': 'item'}, inplace=True)
dataset.shape

(1635638, 3)

In [412]:
dataset_dict = dataset.to_dict()

In [413]:
dataset.columns

Index(['id_ученика', 'возраст', 'item'], dtype='object')

In [414]:
interests = [[key, val] for key, val in dataset['item'].value_counts().to_dict().items()]
total = sum([val for key, val in interests])
interests = pd.DataFrame([[key, val / total] for key, val in interests])

begin = max(dataset_dict['item']) + 1

for row in tqdm(dataset.itertuples(), total=len(dataset)):
    current_idx = begin + row[0]
    names = np.random.choice(interests[0], 2, p=interests[1])
    name = names[0] if names[0] != row[3] else names[1]
    dataset_dict['item'].update({current_idx: name})
    dataset_dict['возраст'].update({current_idx: row[2]})
    dataset_dict['id_ученика'].update({current_idx: row[1]})

    
#interests[0]

100%|██████████| 1635638/1635638 [02:11<00:00, 12412.56it/s]


In [415]:
dataset = pd.DataFrame.from_dict(dataset_dict)
dataset['label'] = [1] * int(len(dataset) / 2) + [0] * int(len(dataset) / 2)
dataset.shape

(3271276, 4)

In [416]:
dataset.dropna(inplace=True)
dataset.shape

(3271276, 4)

In [417]:
dataset['возраст'] = pd.qcut(dataset['возраст'], q=5).astype(str)

In [418]:
cat_cols = [
    'возраст'
]

for col in tqdm(cat_cols):
    dummy = pd.get_dummies(dataset[col].astype(str), dummy_na=True)
    dummy.columns = [str(col) + str(c) for c in dummy.columns]
    del dataset[col]
    dataset = pd.concat([
        dataset,
        dummy
    ], axis=1)

100%|██████████| 1/1 [00:00<00:00,  1.82it/s]


In [419]:
dataset.shape

(3271276, 9)

In [420]:
user_map = {iD: i for i, iD in enumerate(dataset['id_ученика'].unique())}
item_map = {iD: i for i, iD in enumerate(dataset['item'].unique())}

dataset['id_ученика'] = dataset['id_ученика'].map(user_map)
dataset['item'] = dataset['item'].map(item_map)

In [421]:
class RcDataset(Dataset):
    
    def __init__(self, dataset):
        self.data_cols = dataset.columns.to_list()
        self.data_cols.remove('label')
        self.data_len = len(dataset)
        
    def __len__(self):
        return self.data_len
    
    def __getitem__(self, idx):
        sample = dataset.iloc[idx]
        input_ = sample[self.data_cols].to_numpy()
        #input_ = torch.FloatTensor(input_)
        label = np.float32(sample['label'])
        #label = torch.LongTensor(sample['label'])
        #if label == 1:
        #    label = torch.LongTensor([0, 1])
        #else:
        #    label = torch.LongTensor([1, 0])

        return input_, label

In [422]:
from sklearn.model_selection import StratifiedKFold

#skf = StratifiedKFold(n_splits=20)
#train_index, test_index = next(skf.split(dataset, dataset['label']))

alpha = 0.05
dataset = dataset.sample(frac=1).reset_index(drop=True)
test_len = int(len(dataset) * alpha)

dataset_train = RcDataset(dataset[:-test_len])
dataloader_train = DataLoader(dataset_train, batch_size=100, shuffle=True)

dataset_test = RcDataset(dataset[-test_len:])
dataloader_test = DataLoader(dataset_test, batch_size=100, shuffle=True)


In [423]:
n_users = dataset['id_ученика'].unique().shape[0]
n_hobbies = dataset['item'].unique().shape[0]
n_features = dataset.shape[1]-1 
model = RS(n_users, n_hobbies, n_features)

criterion = nn.BCEWithLogitsLoss()
#criterion = nn.CrossEntropyLoss()
sigmoid = nn.Sigmoid()
softmax = nn.Softmax()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
n_epochs = 10
batch_size = dataloader_train.batch_size

In [424]:
for epoch in range(n_epochs):
    for phase in ['train', 'test']:
        running_loss = 0.
        running_acc = 0.
        if phase == 'train':
            dataloader = dataloader_train
            model.train()
        elif phase == 'test':
            dataloader = dataloader_test
            model.eval()
            
        pbar = tqdm(enumerate(dataloader), total=len(dataloader))
    
        for idx, (inputs, labels) in pbar:
            preds = model(inputs)
            preds = softmax(preds)[:,1]
            loss = criterion(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            acc = (preds.round() == labels).float().mean()
            running_acc += acc
            scale_value = 1 / max(idx, 1)
            running_loss += loss.item()
            pbar.set_description(
                "Epoch: {}/{} Phase: {}, Loss: {:.4f}, Acc: {:.4f}".format(
                    epoch,
                    n_epochs,
                    phase,
                    running_loss * scale_value,
                    running_acc * scale_value
            ))

  0%|          | 0/31078 [00:00<?, ?it/s]<ipython-input-424-87f105f5089c>:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = softmax(preds)[:,1]
Epoch: 0/10 Phase: train, Loss: 0.6934, Acc: 0.4999: 100%|██████████| 31078/31078 [1:02:50<00:00,  8.24it/s]
Epoch: 0/10 Phase: test, Loss: 0.6936, Acc: 0.5003: 100%|██████████| 1636/1636 [03:25<00:00,  7.97it/s]
Epoch: 1/10 Phase: train, Loss: 0.6932, Acc: 0.4999: 100%|██████████| 31078/31078 [1:03:56<00:00,  8.10it/s]
Epoch: 1/10 Phase: test, Loss: 0.6936, Acc: 0.5003: 100%|██████████| 1636/1636 [03:23<00:00,  8.05it/s]
Epoch: 2/10 Phase: train, Loss: 0.6932, Acc: 0.4999: 100%|██████████| 31078/31078 [1:05:10<00:00,  7.95it/s]
Epoch: 2/10 Phase: test, Loss: 0.6936, Acc: 0.5003: 100%|██████████| 1636/1636 [03:26<00:00,  7.91it/s]
Epoch: 3/10 Phase: train, Loss: 0.6932, Acc: 0.4999: 100%|██████████| 31078/31078 [1:06:19<00:00,  7.81it/s]
Epoch: 3/10 Phase: test

KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.lib.c_is_list_like'
Traceback (most recent call last):
  File "/home/konstantinlipkin/anaconda3/lib/python3.8/abc.py", line 96, in __instancecheck__
    def __instancecheck__(cls, instance):
KeyboardInterrupt: 
Epoch: 5/10 Phase: train, Loss: 0.6943, Acc: 0.5035:   2%|▏         | 596/31078 [01:19<1:07:44,  7.50it/s]


KeyboardInterrupt: 